<a href="https://colab.research.google.com/github/Jamar-Mitchell/AE-Applications-Web-App/blob/master/DS_Workshop_Ht6_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the Toronto Open311 Dataset

By [Matthew Reyes](https://www.linkedin.com/in/matthewnoahreyes/) \\
Co-founder @ Hypotenuse Labs \\
matt@hypotenuse.ca

This workshop focuses on exploring data with Python.

The dataset we'll be looking at is from the [Toronto Open311 API](https://www.toronto.ca/home/311-toronto-at-your-service/). Open311 is an open standard for civic issue tracking, and has been used in cities such as Toronto, Chicago, and Boston.

The main library we'll use is `pandas`, a Python library for data manipulation and analysis. We'll also import a few other useful libraries here.


In [0]:
import numpy as np
import pandas as pd
import json
import requests
import datetime as dt
from datetime import datetime
import dateutil.parser
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 100)

## Download the Dataset

Let's read the dataset into a table.

In the `pandas` library, we'll use the *DataFrame* object. A DataFrame is a 2D tabular data structure - we'll see shortly how we can aggregate and summarize it.

For this workshop's convenience, I've already retrieved about 10 years worth of service requests. More details on how this was done in the Appendix section of this notebook.

Before proceeding, it might be helpful to take a look at our data's [documentation](https://open.toronto.ca/dataset/311-open311-api-calls-for-service-requests/).

In [0]:
!wget https://github.com/mattreyes/toronto-open311-data/raw/master/toronto311_2009_2018.csv

In [0]:
%ls

Let's try to see what the file looks like.

In [0]:
# Print out the first 10 rows
import csv
with open('toronto311_2009_2018.csv') as csvfile:
    reader = csv.reader(csvfile)
    for i, row in enumerate(reader):
        print(row)
        if(i >= 9):
            break

Thankfully, pandas has a much faster way of reading an entire CSV dataset at once. We save the data to the variable `df` - a Pandas data structure.

In [0]:
df = pd.read_csv('toronto311_2009_2018.csv')

## Selecting Data

With data loaded into our Colab filesystem, we can begin to profile it.

We can look at the 'head' or 'tail' of the dataset - the first or last $k$ rows of the dataset, respectively.

In [0]:
df.head(5)

In [0]:
df.tail(3)

In [0]:
# How many rows and columns does df have?
df.shape

In [0]:
# What columns does df have?
df.columns

### Column Selection

To select a single column from a dataframe:

In [0]:
df['address']

In [0]:
# To select a subset of columns, pass in a list of column names
df[['address', 'address_id']]

In [0]:
# The list can be a variable
some_cols = ['service_request_id', 'requested_datetime', 'service_name', 'address']

df[some_cols]

In [0]:
# Saving the subset of df to a variable
df_some_cols = df[some_cols]
df_some_cols

Exercise:

1. Select the `requested_datetime` column. What do you see?
2. Select the `description` column. What do you see?

In [0]:
# EXERCISE 1

# YOUR CODE HERE

In [0]:
# EXERCISE 2

# YOUR CODE HERE

### Row Selection
A general formula for selecting a subset of rows: `DataFrame[boolean_expression]`

In [0]:
# Example of a boolean array
df['service_name'] == 'Road - Pot hole'

In [0]:
# Select rows where service_name is 'Road - Pot hole'
criteria = df['service_name'] == 'Road - Pot hole'

df[criteria]

In [0]:
# Select incomplete requests

criteria2 = df['status_notes'] != 'Completed - The request has been concluded.'

df[criteria2]

In [0]:
# Aside: selection of rows from anywhere

# Select by index
df_aside = df.loc[100:105]

# Select by row number
df.iloc[-1]

df_aside.reset_index()

##### Exercises

1. Select all requests which have been closed.
2. Select all requests which aren't related to pot holes.
3. Select all requests which are missing an address. Hint: use the `isnull()` method on a column.

In [0]:
# EXERCISE 1

# YOUR CODE HERE

In [0]:
# EXERCISE 2

# YOUR CODE HERE

In [0]:
# EXERCISE 3

# YOUR CODE HERE

### Counting Nulls

Based on looking at a few rows, a lot of columns can contain *null* values. Let's get an idea of how many null values are in each column. 

In Python, null values might look like `NaN` (not a number) or `None`. Null values might occur if we are missing info for that particular field, or if that column doesn't apply to it.



In [0]:
df.info()

Viewing all the unique values a column takes on can also be informative.

In [0]:
df['service_name'].unique()

Exercises:

1. Look at the unique list of values for `status_notes`. What do you notice?
2. Look at the unique list of values for `agency_responsible`. What do you notice?
3. Pick any other column and look at the list of unique values. What do you notice?


In [0]:
# EXERCISE 1

# YOUR CODE HERE

In [0]:
# EXERCISE 2

# YOUR CODE HERE

In [0]:
# EXERCISE 3

# YOUR CODE HERE

## Converting a column of date strings

The three columns with a timestamp are actually strings. If we want to perform operations with them we'd need to convert them to a `datetime` type.

In [0]:
df['requested_datetime'].head()

These timestamps are actually ISO8601 timestamps. More info on these [here](https://www.w3.org/TR/NOTE-datetime).

For example, `1994-11-05T08:15:30-05:00` corresponds to November 5, 1994, 8:15:30 am, US Eastern Standard Time.

In [0]:
# Example: converting 'requested_datetime' to a datetime type

pd.to_datetime(df['requested_datetime'][9000:10000]).dt.tz_convert('US/Eastern')

In [0]:
df.head(2)

## Adding columns

Silly example: adding a new column called `'lmao'` consisting of all 1's:

In [0]:
df['lmao'] = 1
df.head(10)

This is a pretty useless column so let's drop it.

In [0]:
df.drop(columns='lmao', inplace=True)

In [0]:
df.head(2)

In [0]:
df.drop(columns='agency_responsible', inplace=True)

In [0]:
df.columns

A more practical example: add in the 3 converted timestamp columns.

In [0]:
# Timestamp conversions
df['requested_datetime_conv'] = pd.to_datetime(df['requested_datetime'], utc=True).dt.tz_convert('US/Eastern')
df['expected_datetime_conv'] = pd.to_datetime(df['expected_datetime'], utc=True).dt.tz_convert('US/Eastern')
df['updated_datetime_conv'] = pd.to_datetime(df['updated_datetime'], utc=True).dt.tz_convert('US/Eastern')
# Dropping rid of any erroneous columns (this step is needed for later)
df = df[df['requested_datetime_conv'].notnull()]

Now for closed requests, we can calculate how long it took to close (in hours).

When we subtract two dates we get a `timedelta` object. We then get the total number of seconds between those two dates and convert to **number of days**.

In [0]:
df_closed = df[df['status'] == 'closed']

df['actual_completion_time'] = (df_closed['updated_datetime_conv'] - df_closed['requested_datetime_conv']).dt.total_seconds()/60/60/24
df['est_completion_time'] = (df_closed['expected_datetime_conv'] - df_closed['requested_datetime_conv']).dt.total_seconds()/60/60/24

In [0]:
df[df['status'] != 'closed'].head(3)

In [0]:
df[df['status'] == 'closed'].head(3)

##### Exercise:

Add a column called `dayofweek` that encodes the day-of-week that the request occurred. (i.e. Mon, Tues, Sun) You'll probably need to search online for the appropriate method.

In [0]:
# EXERCISE

# YOUR CODE HERE

## Data Summarization & Aggregation

Much like Excel, we can perform operations such as *summations, averages, and counting* on the rows of a table.

In [0]:
# Note that count() only counts non-null values
df.count()

The `.describe()` method gives us a quick way of doing these sorts of aggregations all at once.

In [0]:
# Get a 5-number summary of the data
df.describe()

### Grouping

When aggregating our data, we often want to group it by categorical variables. For example, if you wanted to count the number of requests for each type of service request.

For SQL users, this is analogous to querying a table using a `GROUP BY` clause and aggregation functions such as `SUM()` or `COUNT()`. For Excel users, this could be done using a pivot table.

In pandas, we have the `.groupby()` method which we can apply aggregation functions to.

More reading: https://pandas.pydata.org/pandas-docs/stable/groupby.html

In [0]:
df.groupby(by='status')

In [0]:
df[['address', 'description', 'status']].groupby(by='status').count()

In [0]:
df.groupby(by='status')['address_id'].count()

In [0]:
# Counts non-null and null values. That is, number of rows.
df.groupby(by='status').size()

#### Exercises 
1. How many of each type of request are there? Use the column `service_name`.
2. How many requests were made on each day of the week? (Mon, Tues, Wed, etc.).
3. What's the average actual completion time for each type of request?
4. (BONUS) Select all requests made in the month of January 2017. How many types of requests were made this month?
5. (BONUS) Double-check if the `service_request_id` is really a unique identifier for each request.

In [0]:
# EXERCISE 1

# YOUR CODE HERE

In [0]:
# EXERCISE 2

# YOUR CODE HERE

In [0]:
# EXERCISE 3

# YOUR CODE HERE

## Plotting

In [0]:
# Importing some plotting libraries

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")


In [0]:
# Histograms
df['actual_completion_time'].plot.hist(bins=100)
plt.xlim(0,500)

In [0]:
# Boxplots
# More info on boxplots here: https://math.stackexchange.com/questions/2172520/what-is-difference-between-fliers-ans-whisker-in-boxplot
df['actual_completion_time'].plot.box(showfliers=False)

plt.ylabel('Actual completion time (days)')
plt.show()

In [0]:
df['est_completion_time'].plot.box(showfliers=False)

In [0]:
df[df['service_name'] == 'Bridge - Graffiti Complaint']['actual_completion_time'].hist()

In [0]:
df[df['service_name'] == 'Bridge - Graffiti Complaint']['est_completion_time'].plot.box(showfliers=False)

In [0]:
# Bar plots
df.groupby(by=['service_name']).size().sort_values(ascending=False).plot.bar()


In [0]:
# For temporal variables we can aggregate (or "resample") the data to different time periods, such as day, week, and month.
# More on this: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Grouper.html
df.groupby(pd.Grouper(key='requested_datetime_conv', freq='W')).size().plot(figsize=(15,5), marker='.')
plt.title('Total service requests per week')
plt.show()

In [0]:
# Conditional boxplots
df.boxplot(column='actual_completion_time', by='service_name', showfliers=False, rot=45)

# df[['actual_completion_time', 'est_completion_time', 'service_name']].boxplot(showfliers=False, by='service_name', rot=30)

#### Exercises 

1. Plot a histogram of the `est_completion_time` column. What do you notice?
2. For a boxplot, what happens if you don't include the argument `showfliers=False`?
3. Plot the average `actual_completion_time`, grouped by a date field like month, hour of day, or year.


In [0]:
# EXERCISE 1

# YOUR CODE HERE

In [0]:
# EXERCISE 2

# YOUR CODE HERE

In [0]:
# EXERCISE 3

# YOUR CODE HERE

## Asking & Answering Questions

One important aspect of the data science process is asking good questions. During our simple exploration, you may have already come up with some questions.

After this workshop, I encourage you to try and answer some of these with the information we have. 

In practise, the data will have quirks and business logic that you will need to ask the owners of the dataset about. You'd want to work with stakeholders and decision-makers to prioritize the answers with the highest impact.  

Some ideas brought up during our workshop:

* Can we predict how fast the city will service a request?
  
* How can we measure similarity between incidents?

* Prediction of future requests: where and when will they occur?

* If you don't treat graffiti in a specific area, does it cause more graffiti?
  * Hypothesis: graffiti causes graffiti
  
* Do things get resolved faster if it gets posted on SeeClickFix?
  
* Could this data be useful for building a better service requests triage system?

## (BONUS) Dealing with geospatial data

So far, we haven't touched location-related columns such as `lat`, `long`, `address_id`, and `address`.

Example questions:
- Explore the `address_id` - what exactly does this column do?
- Which areas of the city recieve the most graffiti complaints? Has this changed over the years?
- Which regions had the most service requests? How does that compare with the number of people living there?


In [0]:
df.columns

In [0]:
# What specific lat/long pairs have a lot of service requests?
df.groupby(by=['lat','long']).size().sort_values(ascending=False)

In [0]:
df.groupby(by=['address_id']).size().sort_values(ascending=False)

In [0]:
df.groupby(by=['address']).size().sort_values(ascending=False)

In [0]:
# Drill into each of the requests from these

df[df['address_id'] == 13467983].groupby(by='service_name').size()

In [0]:
# Try plotting the lat/longs on a scatterplot
plt.scatter(df['long'], df['lat'], alpha=0.01)
plt.show()

### Mapping

Let's try to plot these points on a map. We'll use the `folium` library to do this.

In [0]:
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import folium

In [0]:
# Exercise: Try plotting a different location in Toronto

map = folium.Map(location=[43.6532, -79.3832], 
                 zoom_start=14, 
#                  width=500, 
#                  height=500,
                ) 

# Place a marker on the map
folium.Marker([43.659622, -79.397669], popup="Hey that's where we are!").add_to(map)

# Place a circle marker on the map
folium.CircleMarker([43.669978, -79.390157], popup='This is a circle marker', radius=50).add_to(map)

# Display the map
map

In [0]:
# Note: we can use question marks to view docstrings
folium.CircleMarker?

In [0]:
# Plotting all service requests in a single day

date_range_mask = (df['requested_datetime_conv'] >= '2016-08-16') & (df['requested_datetime_conv'] < '2016-08-17')
df_oneday = df[date_range_mask]
map2 = folium.Map(location=[43.6532, -79.3832], zoom_start=14) 
  
# Loop through each row, add a marker
for idx, r in df_oneday.iterrows():
  folium.CircleMarker([r['lat'], r['long']], popup=r['service_name'], radius=5).add_to(map2)
  
map2

In [0]:
# Plotting all service requests in a month

date_range_mask = (df['requested_datetime_conv'] >= '2016-08-16') & (df['requested_datetime_conv'] < '2016-08-24')
df_onemonth = df[date_range_mask]
map3 = folium.Map(location=[43.6532, -79.3832], zoom_start=14) 
  
# Loop through each row, add a marker
for idx, r in df_onemonth.iterrows():
  folium.CircleMarker([r['lat'], r['long']], radius=25, fill_color='green', color='green',  popup=r['service_name']).add_to(map3)
  
map3

In [0]:
# Markerclusters are cleaner for many markers
from folium.plugins import MarkerCluster

map4 = folium.Map(location=[43.6532, -79.3832], zoom_start=13) 
  
marker_cluster = MarkerCluster().add_to(map4)

# Loop through each row, add a marker
for idx, r in df_onemonth.iterrows():
  folium.CircleMarker([r['lat'], r['long']], radius=15, fill_color='green', color='green',  popup=r['service_name']).add_to(marker_cluster)
  
map4

In [0]:
date_range_mask = (df['service_name'] != 'Road - Pot hole') & (df['requested_datetime_conv'] >= '2016-09-01') & (df['requested_datetime_conv'] < '2016-12-31')
df_graffiti = df[date_range_mask]

In [0]:
# Heatmaps are pretty :)
# However, interpret them with care! https://xkcd.com/1138/

hm = folium.Map(location=[43.6532, -79.3832], zoom_start=13) 

# List comprehension to make a list of lists
request_pts = [[row['lat'], row['long']] for index, row in df_graffiti.iterrows()]


HeatMap(request_pts).add_to(hm)


# Display the map
hm

Future work:
- How can we divide the city into different regions and categorize requests by area?
- Which areas take longest to service?
- Can we make a heatmap of request volume over time?


 ## Looking ahead

[More info](https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#e2634d40-0dbf-4d91-12fa-83f039307e93) on the Open311 API.

You might want to run notebooks like this one on your local machine. These can be set up locally as [Jupyter Notebooks](https://jupyter.org/).

Learn how to [automating boring, tedious tasks with Python.](https://automatetheboringstuff.com/)

[A fast Python reference](https://learnxinyminutes.com/docs/python/)

A [style guide](https://www.python.org/dev/peps/pep-0008/) for writing Python code.

[Codecademy](https://www.codecademy.com/catalog/language/python) is a fantastic place to learn Python essentials.

A [quick primer](https://pandas.pydata.org/pandas-docs/stable/10min.html) on the pandas library.

Some [clarification](https://stackoverflow.com/questions/26047209/what-is-the-difference-between-a-pandas-series-and-a-single-column-dataframe) on the difference between a `Series` and a `DataFrame` object.  

## Next steps

- Other cities may have Open311 datasets for the public to view - [Edmonton](https://data.edmonton.ca/City-Administration/311-Requests/q7ua-agfg), for example. Try building your own analysis from scratch! 
- Try collecting data from another API (such as Twitter, Facebook, Github, etc.) and bringing it into a Jupyter or Colab notebook.
- Check out another dataset of Toronto service requests [here](https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#2fc027de-d27a-2a00-1388-6849497cb84c). It seems to contain a variety of other service request types we don't see in this notebook's dataset, including "Watermain-Possible Break", "Litter / Illegal Dumping Cleanup", and "CADAVER WILDLIFE".

## Appendix - The Open311 API


This section documents how the dataset was collected.

##### Exercise 1 

Let's try calling the Open 311 API directly through our browser. Copy and paste this URL into your browser: 

`https://secure.toronto.ca/webwizard/ws/requests.json?start_date=2012-02-01T00:00:00Z&end_date=2012-02-07T00:00:00Z&jurisdiction_id=toronto.ca`

What do you see? What do you think this URL does?

##### Exercise 2

*SeeClickFix* is a web tool used for citizens to report non-emergency neighbourhood issues to their local government. To see issues reported for Toronto, go here:

- https://seeclickfix.com/can_toronto

Scroll through the various issues being reported. Some issues are assigned Request ID's by the user "Open311 Toronto". A couple of examples:

- https://seeclickfix.com/issues/5283320-derelict-abandoned-bicycle-danforth-ave-e-of-playter
- https://seeclickfix.com/issues/5272857-king-st-e-and-sherbourne-st-pedestrian-crossing-signal-countdown-missing-failed

Let's view these issues directly through the API. Similarly to *Exercise 1*, pick one of these two links and make a request for that issue.

Hint: We won't need the `start_date` or `end_date` arguments in our query string. But we'll need `jurisdiction_id` and `service_request_id`.

Something to note for later: "Please note that we only have graffiti and pothole templates to choose from in the mobile app, so are forced to submit non-pothole issues as Graffiti on a City Road, as this template is known to go to Transportation Services. Thank you."

### Python Requests

Now let's make the same query programmatically in Python, via the `requests` library.

In [0]:
ENDPOINT = "https://secure.toronto.ca/webwizard/ws/requests.json?"

# Using Exercise 2's parameters
params = {
            "service_request_id": "101005679196",
            "jurisdiction_id":"toronto.ca",
         }

# Make a GET request and save the response to a variable
resp = requests.get(ENDPOINT, params=params)

# View the URL it used to make the request
resp.url

In [0]:
# Get the response's text
resp.text

In [0]:
# Decode the response's text to JSON.
resp_json = resp.json()
resp_json

##### Exercise 3

Modify the above cells to make the same request as Exercise 1.

In [0]:
params_ex1 = {
            "start_date": "2012-02-01T00:00:00Z",
            "end_date": "2012-02-07T00:00:00Z",
            "jurisdiction_id":"toronto.ca",
         }


resp = requests.get(ENDPOINT, params=params_ex1)

resp.json()

##### Exercise 4

Get all service requests between Mar 1 2018 and Mar 5 2018, inclusive.


In [0]:
params_ex4 = {
            "start_date": "2012-03-01T00:00:00Z",
            "end_date": "2012-02-07T00:00:00Z",
            "jurisdiction_id":"toronto.ca",
         }


resp = requests.get(ENDPOINT, params=params_ex1)

resp.json()

##### Exercise 5

Try retrieving all service requests made in 2018. How many were made? Does this number make sense?

In [0]:
# If you try this, you'll find that only 1000 requests in total are returned.

### Dealing with API limitations

According to the Toronto Open311 documentation, responses can only return up to 1000 service requests per API call. If we want to collect all service requests in a wide date range (say, in *Exercise 5*), this is an issue.

Other APIs will likely require authentication from the user via an API token, usually passed as a parameter in the request. They may also have limitations on the number of requests you can make per second/hour/etc.

##### Bonus reading
- http://www.open311.org/
- https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#e2634d40-0dbf-4d91-12fa-83f039307e93

One way to get around a response-size limitation is to break our date range into N contiguous subintervals and make multiple API calls for each one. How can we do this programmatically?





### Working with date strings

If you've used Excel before, dates are actually processed as numbers. This allows easy date operations such as 
- Adding one hour/day/month/etc. to a date
- sorting by date
- finding the time difference between two dates

Similarly, Python has the `datetime` module for working with dates/times.

In [0]:
def date_range(start, end, N, tz_string="-05:00"):
    """Splits a given date range into N contiguous subintervals in ISO8601 timestamp format. 
    
    start and end must be strings in "YYYYMMDD" format.
    tz_string assumed to be Toronto timezone.
    
    """
    start = datetime.strptime(start,"%Y%m%d")
    end = datetime.strptime(end,"%Y%m%d")
    diff = (end  - start ) / N
    dates = []
    for i in range(N):
        dates += [(start + diff * i).strftime("%Y-%m-%dT%H:%M:%S") + tz_string]
    dates += [end.strftime("%Y-%m-%dT%H:%M:%S") + tz_string]
    return dates

In [0]:
# Trying out our date_range function
dates = date_range('20180301', '20180305', 7)
dates

Next, we'll to loop over this list and call the API for each interval. For each call we'll add the results to the `service_requests` list.



In [0]:
service_requests = []

for i, d in enumerate(dates[:-1]):
    curr_params = {
        "start_date": dates[i],
        "end_date": dates[i+1],
        "jurisdiction_id":"toronto.ca"
     }
    resp = requests.get(ENDPOINT, params=curr_params)
    resp_json = resp.json()['service_requests']
    service_requests += resp_json
    print("Step {0}: {1} to {2}, number of results: {3}".format(i, dates[i], dates[i+1], len(resp_json)))

In [0]:
# View 3 examples
service_requests[:3]

##### Extra Credit 

- How can we make our `fetch_data` function more robust? Say, if a particular request fails? Or if the request returns None?

- After this workshop, try collecting 3 years' worth of requests. You might want to save your intermediate results to a file as you iterate through your date range, in case your notebook crashes or some requests fail. More info on file I/O: https://colab.research.google.com/notebooks/io.ipynb



#### Saving the data to a DataFrame

In [0]:
# Encode our list into a JSON string
json_str = json.dumps(service_requests)

# Read the JSON string into a DataFrame
df = pd.read_json(json_str, orient='records')

df